In [34]:
import requests
from glob import glob
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import time

In [35]:
headers= ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})

url = pd.read_csv('List of URL.csv', sep=',')

book_list = []

In [39]:
dataframe = pd.read_csv('List of URL.csv', sep=',')

def book_function(url_data):
    

    data=[]

    for url in url_data['URL']: #[:4]: slicing only the top 4 
        time.sleep(3)
        amazon = requests.get(url, headers=headers)
        content = amazon.content
        soup = BeautifulSoup(content)

        title = soup.find(id='productTitle').get_text().strip()

        # to prevent script from crashing when there isn't a price for the product
        try:
            price =(soup.find('span', attrs = {'class':'a-size-base a-color-price a-color-price'}).get_text()).strip().replace('$', '')
        except:
            #price = 'Nothing'
            try:
                price =(soup.find('span', attrs = {'class':'slot-price'}).get_text()).strip().replace('$', '')
            except:
                try:
                    price = (soup.find('span', attrs = {'class':'a-size-base a-color-secondary'}).get_text()).strip().replace('$', '')
                #price = 'Nothing'
                except:
                    price = 'Nothing'
                

        try:
            review_score = float(soup.select('.a-star-4-5')[0].get_text().split(' ')[0].replace(",", "."))
        except:
            #review_score = 'Nothing'
            try:
                review_score = float(soup.select('i[class*="a-icon a-icon-star a-star-"]')[1].get_text().split(' ')[0].replace(",", "."))  
            except:
                try:
                    review_score = float(soup.select('a-size-medium a-color-base a-text-beside-button a-text-bold')[1].get_text().split(' ')[0].replace(",", "."))
                except:
                    review_score = 'Nothing'
                    
                #review_score = 'Nothing'
            
        try: 
            review_count = int(soup.select('#acrCustomerReviewText')[0].get_text().split(' ')[0].replace(",", ""))
        except:
    
            review_count =  'Nothing'
            #try:
            #    review_score = float(soup.select('i[class*="a-icon a-icon-star a-star-"]')[1].get_text().split(' ')[0].replace(",", "."))
            #    review_count = int(soup.select('#acrCustomerReviewText')[0].get_text().split(' ')[0].replace(".", ""))
            #except:
            #    review_score = ''
            #    review_count = ''

        data.append({'Title': title, 'Price': price, 'url':url, 'Review_score':review_score, 'Review_count': review_count })
    
    return pd.DataFrame(data)

In [40]:
booklist = book_function(dataframe)
booklist.head()

,Title,Price,url,Review_score,Review_count
0,The Last Thing He Told Me: A Novel,16.88,https://www.amazon.com/dp/1501171348?tag=NYTBS...,4.4,7509
1,Sooley: A Novel,17.37,https://www.amazon.com/dp/0385547684?tag=NYTBS...,4.5,10868
2,Project Hail Mary: A Novel,15.89,https://www.amazon.com/dp/0593135202?tag=NYTBS...,4.7,8988
3,Where the Crawdads Sing,10.99,https://www.amazon.com/Where-Crawdads-Sing-Del...,4.8,133495
4,"21st Birthday (Women's Murder Club, 21)",17.69,https://www.amazon.com/dp/031649934X?tag=NYTBS...,4.5,8249


In [30]:
#booklist = book_function(dataframe)

In [41]:
booklist.to_csv('List of URL_scraping.csv', sep = ';', index=True)